# Step 1: Environment Setup 

Welcome to the Illinois System Simulation Toolkit (ISST) for AE 443! This notebook will walk you through connecting the various technical, cost, and schedule analyses necessary for your design project. First, let's check that you have all the necessary libraries installed.

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import pymc as pm

from scipy.optimize import curve_fit

import pint
ureg = pint.UnitRegistry()
ureg.default_format = '~P'
ureg.define('EUR = [currency]')
ureg.define('USD = 0.91 * EUR')

If the above cell triggered an error, you're most likely missing some of the required libraries. You can either install them yourself, or run the following cell to install them.

In [ ]:
# import sys
# !{sys.executable} -m pip install -U numpy pandas matplotlib arviz pymc pint

Once you have the required libraries installed and imported, you can proceed to the next step.

# Step 2: Creating the ISST Model Context

Throughout this notebook, we'll be building up a PyMC model context for relating all the parts of your design system together. As such, whenever you want to create a new variable or element of the overall design system, you'll do it within the context:

In [68]:
ISST_Model_Context = pm.Model()

with ISST_Model_Context:
    P1 = pm.Normal('Parameter Name', mu=0, sigma=1)

In the above cell, we've created a parameter, and characterized our uncertainty about it as a normal distribution with mean 0 and standard deviation 1. Note that it has both a variable name 'P1' which will be used to refer to it throughout the code, but it also has a plain language reference to it, 'Parameter Name'. The tag is what will show up in the charts and other reports that will be generated by ISST, so feel free to mix and match your variable names as makes it easiest for you to read the code, while using the tags to determine how they'll show up outside of the code.

Because the parameter was created in the ISST model context (by indenting it beneath the `with ISST_Model_Conxtext` line), ISST knows that the parameter is a part of the overall design system, and it will keep track of its relationship to other parts of the design system.

# Step 3: Creating Risk Tables

ISST wraps around PyMC models, asking you for just a few plain language properties of your design system. Below is the definition of the Risk Table Class:

In [106]:
from dataclasses import dataclass, field

@dataclass
class Risk_Table:
    
    # Name of the Risk Table
    name: str = field(init=True)
    
    # Units of the impact the Risk Table tracks, e.g. 'kg/m^2' using pint's unit registry:
    # https://pint.readthedocs.io/en/stable/user/formatting.html 
    severity_units: str = field(init=True)
    
    # Breakpoints
    severity_level_breakpoints: list[float] = field(init=True)
    
    # Utility Of Each Severity Level in the Risk Table. Defaults to [-1, -3, -5, -9]
    severity_utilities: list[float] = field(init=True, default=None)
        
    # Discrete Likelihood of Each Severity Level in the Risk Table. Defaults to geometrics means (sqrt(A*B))of ICD-203 Levels
    probability_level_nominals: list = field(init=True, default_factory=lambda: [0.022, 0.100, 0.300, 0.497, 0.663, 0.87, 0.97])
    
    probability_level_descriptions: list[str] = field(init=True, default_factory=lambda:['Remote',
                                                                                         'Very Unlikely',
                                                                                         'Unlikely',
                                                                                         'Roughly Even Chance',
                                                                                         'Likely',
                                                                                         'Very Likely',
                                                                                         'Almost Certain'])
    
    L : float = field(init=True, default=10.0)
    k : float = field(init=True, default=1.0)
    x0: float = field(init=True, default=0.0)
    
    def __post_init__(self):
        if self.severity_level_breakpoints[0] != 0.:
            self.severity_level_breakpoints.insert(0, 0.)
        if self.severity_utilities is None:
            self.severity_utilities = np.linspace(0, -9, len(self.severity_level_breakpoints)).tolist()
        logistic_params = self.fit_utilities()
        self.L = logistic_params[0][0]
        self.k = logistic_params[0][1]
        self.x0 = logistic_params[0][2]

            
    def _utility(self,
                impact,
                L,
                k,
                x0,
                mode = 'logistic',
                u_func = lambda x:x):
        if mode == 'discrete':
            return self.severity_utilities[np.max(np.where(np.array(self.severity_level_breakpoints) < impact)) + 1]
        if mode == 'logistic':
            return self.L/(1 + np.exp(-k*(impact - x0)))
        if mode == 'custom':
            return u_func(impact)
    
    def fit_utilities(self):

        u_func = lambda impact, L, k, x0:self._utility(impact, mode='logistic',L=L, k=k, x0=x0)
        params = curve_fit(u_func, xdata=np.array(self.severity_level_breakpoints), ydata=np.array(self.severity_utilities))
        
        return params

    def discrete_utility(self, impact):
        return self._utility(impact, mode='discrete')
    
    def logistic_utility(self, impact):
        return self._utility(impact, mode='logistic', L = self.L, k = self.k, x0 = self.x0)
    
    def custom_utility(self, impact, utility_function):
        return self._utility(impact, mode='custom', u_func = utility_function)

Schedule_Risk_Table = Risk_Table(name='Schedule',
                                 severity_units='months',
                                 severity_level_breakpoints=[0., 1., 6., 24.,])

/home/jordan/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Note that the breakpoints are the dividing lines between the levels of risk impact, and as a result there are only three breakpoints to define four levels of risk.

In [107]:
from pprint import pprint as pp
pp(Schedule_Risk_Table)

Risk_Table(name='Schedule',
           severity_units='months',
           severity_level_breakpoints=[0.0, 1.0, 6.0, 24.0],
           severity_utilities=[0.0, -3.0, -6.0, -9.0],
           probability_level_nominals=[0.022,
                                       0.1,
                                       0.3,
                                       0.497,
                                       0.663,
                                       0.87,
                                       0.97],
           probability_level_descriptions=['Remote',
                                           'Very Unlikely',
                                           'Unlikely',
                                           'Roughly Even Chance',
                                           'Likely',
                                           'Very Likely',
                                           'Almost Certain'],
           L=1.0,
           k=-5.159227228921725,
           x0=-3.443032392208005)


Let's say we want to adjust the severity levels of the schedule risk table. We think the 6-24 month category is too broad, and want to split it into two categories:

In [108]:
Schedule_Risk_Table.severity_level_breakpoints = [1., 6., 12., 24.]
Schedule_Risk_Table.severity_levels = [-1, -3, -5, -7, -9]
pp(Schedule_Risk_Table)

Risk_Table(name='Schedule',
           severity_units='months',
           severity_level_breakpoints=[1.0, 6.0, 12.0, 24.0],
           severity_utilities=[0.0, -3.0, -6.0, -9.0],
           probability_level_nominals=[0.022,
                                       0.1,
                                       0.3,
                                       0.497,
                                       0.663,
                                       0.87,
                                       0.97],
           probability_level_descriptions=['Remote',
                                           'Very Unlikely',
                                           'Unlikely',
                                           'Roughly Even Chance',
                                           'Likely',
                                           'Very Likely',
                                           'Almost Certain'],
           L=1.0,
           k=-5.159227228921725,
           x0=-3.443032392208005)


# Step 4: Creating Risks

Now that we have a risk table, we can create begin to specify risks. Similarly to Risk Tables, Risks are a custom class for ISST. They're defined primarily by the Risk Tables associated with them. Each risk has its associated schedule and cost risk tables, but also a LIST of technical risk tables:

In [54]:
@dataclass
class Risk:
    
    # Name of the Risk
    name: str = field(init=True)
    
    # Schedule Risk Table
    schedule_risk_table: Risk_Table = field(init=True, default=None)
    
    # Technical Risk Tables associated with the Risk
    technical_risk_tables: list[Risk_Table] = field(init=True, default=None)
    
    # Cost Risk Table
    cost_risk_table: Risk_Table = field(init=True, default=None)
    

This might seem a little strange - adding the risk tables as attributes of the risks, but keep in mind that Python passes attributes by reference by default. This allows us to keep all of our risks updated if there's a change in the risk table. For instance, let's create a "Shipping Delay Risk":

In [58]:
R1 = Risk(name='Shipping Delay Risk', schedule_risk_table=Schedule_Risk_Table)

If we check the severity level breakpoints of the Risk's schedule risk table, we get the expected `[1.0, 6.0, 12.0, 24.0]`:

In [59]:
R1.schedule_risk_table.severity_level_breakpoints

[1.0, 6.0, 12.0, 24.0]

If we now add an extra level to the breakpoints of Schedule_Risk_Table and check R1's breakpoints again:

In [60]:
Schedule_Risk_Table.severity_level_breakpoints=[1.0, 6.0, 12.0, 24., 48.]
R1.schedule_risk_table.severity_level_breakpoints

[1.0, 6.0, 12.0, 24.0, 48.0]

We can see that R1's schedule_risk_table.severity_level_breakpoints have now updated, without having to manually reassign them for every risk in the list.

# Step 5: Creating Your System

Now that we've gone over ISST's risk and